In [4]:
import torch
def createDataset(input_size,status_size,output_size,slidingWindow):
    u_seq_total = torch.zeros([0,slidingWindow,input_size])
    y_seq_total = torch.zeros([0,output_size])
    ABCD_seq_total = torch.zeros([0,status_size*status_size+status_size*input_size+output_size*status_size+output_size*input_size])
    length = 5000
    differentMatrix = 1    #生成多少组不同的ABCD
    ABCD_range = length-slidingWindow-1    #每组ABCD用多少个序列
    for j in range(differentMatrix):
        u_seq = torch.zeros([0,slidingWindow,input_size])
        y_seq = torch.zeros([0,output_size])
        ABCD_seq = torch.zeros([0,status_size*status_size+status_size*input_size+output_size*status_size+output_size*input_size])
        A = torch.randn(status_size,status_size)/status_size
        B = torch.randn(status_size,input_size)/status_size
        C = torch.randn(output_size,status_size)
        D = torch.randn(output_size,input_size)

        u = torch.randn(length,input_size)
        x = torch.zeros(status_size,1)

        for i in range(u.size()[0]):
            y = torch.mm(C,x) + torch.mm(D,u[i].view(input_size,1))
            x = torch.mm(A,x) + torch.mm(B,u[i].view(input_size,1))
            u
            if(i>slidingWindow):
                u_seq = torch.cat([u_seq,u[i-slidingWindow:i].view(1,slidingWindow,input_size)])
                y_seq = torch.cat([y_seq,y.view(1,output_size)])
                ABCD = torch.cat([A.flatten(),B.flatten(),C.flatten(),D.flatten()])
                ABCD_seq = torch.cat([ABCD_seq,ABCD.view(1,ABCD.shape[0])])
        u_seq_total = torch.cat([u_seq_total,u_seq])
        y_seq_total = torch.cat([y_seq_total,y_seq])
        ABCD_seq_total = torch.cat([ABCD_seq_total,ABCD_seq])
    return u_seq_total,y_seq_total,ABCD_seq_total,ABCD_range

def createTestDataset(input_size,status_size,output_size,slidingWindow):
    u_seq = torch.zeros([0,slidingWindow,input_size])
    y_seq = torch.zeros([0,output_size])
    ABCD_seq = torch.zeros([0,status_size*status_size+status_size*input_size+output_size*status_size+output_size*input_size])
    length = 500
    differentMatrix = 60     #生成多少组不同的ABCD
    ABCD_range = length-slidingWindow-1    #每组ABCD用多少个序列
    for j in range(differentMatrix):
        A = torch.randn(status_size,status_size)/status_size
        B = torch.randn(status_size,input_size)/status_size
        C = torch.randn(output_size,status_size)
        D = torch.randn(output_size,input_size)
        ABCD = torch.cat([A.flatten(),B.flatten(),C.flatten(),D.flatten()])
        ABCD_seq = torch.cat([ABCD_seq,ABCD.view(1,ABCD.shape[0])])
        u = torch.randn(length,input_size)
        x = torch.zeros(status_size,1)

        for i in range(u.size()[0]):
            y = torch.mm(C,x) + torch.mm(D,u[i].view(input_size,1))
            x = torch.mm(A,x) + torch.mm(B,u[i].view(input_size,1))
            if(i>slidingWindow):
                u_seq = torch.cat([u_seq,u[i-slidingWindow:i].view(1,slidingWindow,input_size)])
                y_seq = torch.cat([y_seq,y.view(1,output_size)])
    return u_seq,y_seq,ABCD_seq,ABCD_range

In [5]:
import torch.nn as nn
class LSTMLinear(nn.Module):
    def __init__(self,input_size,status_size,output_size,slidingWindow):
        super(LSTMLinear,self).__init__()
        self.input_size = input_size
        self.status_size = status_size
        self.output_si
        ze = output_size
        self.slidingWindow = slidingWindow
        self.lstm = nn.LSTM(self.input_size,self.input_size,1,batch_first=True,bidirectional=True,dropout=0.2)
        self.fc1 = nn.Linear(self.input_size*self.slidingWindow*2,self.input_size*self.slidingWindow)
        ABCD = status_size*status_size+status_size*input_size+output_size*status_size+output_size*input_size     #ABCD总变量数
        self.fc2 = nn.Linear(self.input_size*self.slidingWindow*self.output_size,ABCD)
        
    def forward(self,u,y):
        x,_ = self.lstm(u)
        batch,slw,input_feature = x.shape
        x = self.fc1(x.reshape(batch,slw*input_feature))
        x_invy = torch.zeros(batch,slw*self.input_size*self.output_size)
        x_invy = x_invy.cuda()
        for i in range(batch):
            x_invy[i] = torch.mm(torch.pinverse(x[i].view(1,slw*self.input_size)),y[i].view(1,self.output_size)).flatten()
        res = self.fc2(x_invy)
        return res
input_size = 10
status_size = 20
output_size = 10
slidingWindow = 50
assert(input_size == output_size)

(u_seq,y_seq,ABCD_seq,ABCD_range) = createDataset(input_size,status_size,output_size,slidingWindow)
model = LSTMLinear(input_size,status_size,output_size,slidingWindow).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3)

num_epochs = 2000
batch_size = 64
for epoch in range(num_epochs):
    if torch.cuda.is_available():
        u_seq = u_seq.cuda()
        y_seq = y_seq.cuda()
        ABCD_seq = ABCD_seq.cuda()
    else:
        u_seq = u_seq
        y_seq = y_seq
        ABCD_seq = ABCD_seq
    for i in range(int(len(u_seq)/batch_size)):
        u = u_seq[i*batch_size:(i+1)*batch_size]
        y = y_seq[i*batch_size:(i+1)*batch_size]
        # 向前传播
        out = model(u,y)
        loss = criterion(out,ABCD_seq[i*batch_size:(i+1)*batch_size])
    
        # 向后传播
        optimizer.zero_grad() # 注意每次迭代都需要清零
        loss.backward()
        optimizer.step()
    
    if (epoch+1) %1 == 0:
        print('Epoch[{}/{}], loss:{:.6f}'.format(epoch+1, num_epochs, loss.item()))

Epoch[1/2000], loss:0.353659
Epoch[2/2000], loss:0.353593
Epoch[3/2000], loss:0.353527
Epoch[4/2000], loss:0.353461
Epoch[5/2000], loss:0.353395
Epoch[6/2000], loss:0.353330
Epoch[7/2000], loss:0.353264
Epoch[8/2000], loss:0.353198
Epoch[9/2000], loss:0.353133
Epoch[10/2000], loss:0.353068
Epoch[11/2000], loss:0.353002
Epoch[12/2000], loss:0.352937
Epoch[13/2000], loss:0.352872
Epoch[14/2000], loss:0.352807
Epoch[15/2000], loss:0.352743
Epoch[16/2000], loss:0.352678
Epoch[17/2000], loss:0.352613
Epoch[18/2000], loss:0.352549
Epoch[19/2000], loss:0.352484
Epoch[20/2000], loss:0.352420
Epoch[21/2000], loss:0.352356
Epoch[22/2000], loss:0.352292
Epoch[23/2000], loss:0.352228
Epoch[24/2000], loss:0.352164
Epoch[25/2000], loss:0.352100
Epoch[26/2000], loss:0.352037
Epoch[27/2000], loss:0.351973
Epoch[28/2000], loss:0.351910
Epoch[29/2000], loss:0.351846
Epoch[30/2000], loss:0.351783
Epoch[31/2000], loss:0.351720
Epoch[32/2000], loss:0.351657
Epoch[33/2000], loss:0.351594
Epoch[34/2000], los

KeyboardInterrupt: 

In [41]:
(u_test_seq,y_test_seq,ABCD_test_seq,ABCD_range) = createTestDataset(input_size,status_size,output_size,slidingWindow)
u = u_test_seq[0:batch_size]
y = y_test_seq[0:batch_size]
u = u.cuda()
y = y.cuda()
output = model(u,y)



KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
gt_show = [k.cpu().detach().numpy() for k in ABCD_test_seq[0]]
pred_show = [k.cpu().detach().numpy() for k in output[0]]
print(len(gt_show),len(pred_show))
# train_gt_show = [i.cpu().detach().numpy() for k in train_gt for i in k]
# train_pred_show = [i.cpu().detach().numpy() for k in train_pred for i in k]
length = 400
# plt.scatter(list(range(length)), train_gt_show[0:length], c='b')
# plt.scatter(list(range(length)), train_pred_show[0:length], c='r')
plt.scatter(list(range(length)), gt_show[500:length+500], c='b')
plt.scatter(list(range(length)), pred_show[500:length+500
], c='r')
plt.xlabel('point')
plt.ylabel('output')
# plt.title(" system_dim {} output_dim {} delay {}".format( system_dim, output_dim, delay))
plt.show()
plt.close()

In [ ]:
import torch
import sys
from data import get_data, gen_label
from tqdm import tqdm
import torch.nn as nn
import math
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
import cv2

class CLSTMModel(torch.nn.Module):
    def __init__(self,feature_num,slidingWindow,InformationAdd=0,InformationLack=0):
        super(CLSTMModel, self).__init__()
        self.feature_num = feature_num
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(512, self.feature_num)
        )

#         model = models.resnet18(pretrained=True)
# #         model.fc = nn.Identity()
#         self.cnn_layer = model
#         for p in self.parameters():
#             p.requires_grad=False
#         self.cnn_layer.fc = nn.Linear(512,self.feature_num)
        self.slidingWindow = slidingWindow
        self.InformationAdd = InformationAdd
        self.InformationLack = InformationLack
        self.lstm = nn.LSTM(self.feature_num,self.feature_num,1,batch_first=True,bidirectional=True,dropout=0.2)
        self.fc = nn.Linear(self.feature_num*2,1)
        self.slidingFc = nn.Linear(slidingWindow-self.InformationLack+self.InformationAdd,1)
        
        
    def forward(self, u):
        # cnn_x = self.cnn_layer(x)
        # cnn_x = cnn_x.unsqueeze(0)
        x, (h_out, c_out) = self.lstm(u)
        # h = h_out.permute(1, 0, 2)
        # h = h.reshape(h.shape[0], -1)
        # h = self.fc(h)
        
        b,s,h = x.shape
        x = x.reshape(s*b, h) # 转换成线性层的输入格式
        x = self.fc(x)
        x = x.reshape(b,s)
        # x = self.slidingFc(x)
        # x = x.view(b, -1)
        return x
        # return h, h_out, c_out   

slidingWindow = 10
seq_len = 10
InformationAdd = 0
InformationLack = 0
input_size = 10
status_size = 20
output_size = 10
model = CLSTMModel(feature_num, slidingWindow, InformationAdd, InformationLack).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

batch_size = 256
epochs = 10
delay = 10

pbar = tqdm(range(1, epochs+1))


gen_label(system_dim,output_dim,batch_size,False)  # 改变参数需要重新生成label，设reset=True
train_loader, test_loader = get_data(seq_len, batch_size, delay)
for epoch in pbar:
    # h0 = torch.randn(2, batch_size-slidingWindow, feature_num).cuda()
    # c0 = torch.randn(2, batch_size-slidingWindow, feature_num).cuda()
    for batch_index, (images, labels) in enumerate(train_loader):
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        optimizer.zero_grad()
        loss = criterion(outputs.squeeze(0), labels)
        loss.backward()
        pbar.set_postfix({'train loss' : '{0:1.5f}'.format(loss)})
        optimizer.step()


In [1]:
import torch
def createDataset(input_size,status_size,output_size):
    length = 5000
    y_seq = torch.zeros([length,output_size])

    A = torch.randn(status_size,status_size)/status_size
    B = torch.randn(status_size,input_size)/status_size
    C = torch.randn(output_size,status_size)
    D = torch.randn(output_size,input_size)
    u = torch.randn(length,input_size)
    x = torch.zeros(status_size,1)
    for i in range(length):
        y = torch.mm(C,x) + torch.mm(D,u[i].view(input_size,1))
        x = torch.mm(A,x) + torch.mm(B,u[i].view(input_size,1))
        # u_seq = torch.cat([u_seq,u[i-slidingWindow:i].view(1,slidingWindow,input_size)])
        y_seq[i] = y[:,0]
    return u,y_seq

def createTestDataset(input_size,status_size,output_size,slidingWindow):
    u_seq = torch.zeros([0,slidingWindow,input_size])
    y_seq = torch.zeros([0,output_size])
    ABCD_seq = torch.zeros([0,status_size*status_size+status_size*input_size+output_size*status_size+output_size*input_size])
    length = 500
    differentMatrix = 60     #生成多少组不同的ABCD
    ABCD_range = length-slidingWindow-1    #每组ABCD用多少个序列
    for j in range(differentMatrix):
        A = torch.randn(status_size,status_size)/status_size
        B = torch.randn(status_size,input_size)/status_size
        C = torch.randn(output_size,status_size)
        D = torch.randn(output_size,input_size)
        ABCD = torch.cat([A.flatten(),B.flatten(),C.flatten(),D.flatten()])
        ABCD_seq = torch.cat([ABCD_seq,ABCD.view(1,ABCD.shape[0])])
        u = torch.randn(length,input_size)
        x = torch.zeros(status_size,1)

        for i in range(u.size()[0]):
            y = torch.mm(C,x) + torch.mm(D,u[i].view(input_size,1))
            x = torch.mm(A,x) + torch.mm(B,u[i].view(input_size,1))
            if(i>slidingWindow):
                u_seq = torch.cat([u_seq,u[i-slidingWindow:i].view(1,slidingWindow,input_size)])
                y_seq = torch.cat([y_seq,y.view(1,output_size)])
    return u_seq,y_seq,ABCD_seq,ABCD_range

In [2]:
import torch
import sys
from data import get_data, gen_label
from tqdm import tqdm
import torch.nn as nn
import math
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
import cv2

class CLSTMModel(torch.nn.Module):
    def __init__(self,input_size,status_size,output_size,InformationAdd=0,InformationLack=0):
        super(CLSTMModel, self).__init__()
        self.input_size = input_size
        self.status_size = status_size
        # self.cnn_layer = nn.Sequential(
        #     nn.Conv2d(3, 32, kernel_size=3),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=3, stride=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 128, kernel_size=3, stride=2),
        #     nn.ReLU(),
        #     nn.Flatten(),
        #     nn.Linear(512, self.feature_num)
        # )

#         model = models.resnet18(pretrained=True)
# #         model.fc = nn.Identity()
#         self.cnn_layer = model
#         for p in self.parameters():
#             p.requires_grad=False
#         self.cnn_layer.fc = nn.Linear(512,self.feature_num)
        
        self.output_size = output_size
        self.InformationAdd = InformationAdd
        self.InformationLack = InformationLack
        self.lstm = nn.LSTM(self.input_size,self.input_size,1,batch_first=True,bidirectional=True,dropout=0.2)
        self.fc = nn.Linear(self.input_size*2,self.status_size)
        self.fc2 = nn.Linear(self.status_size+input_size,self.output_size)
        self.slidingFc = nn.Linear(slidingWindow-self.InformationLack+self.InformationAdd,1)
        
        
    def forward(self, u):
        # cnn_x = self.cnn_layer(x)
        # cnn_x = cnn_x.unsqueeze(0)
        x, (h_out, c_out) = self.lstm(u.unsqueeze(0))
        # h = h_out.permute(1, 0, 2)
        # h = h.reshape(h.shape[0], -1)
        # h = self.fc(h)
        b,s,h = x.shape
        x = x.reshape(s,h) # 转换成线性层的输入格式

        x = self.fc(x)
        x = torch.cat([x,u],1)
        x = self.fc2(x)
        x = x.reshape(s,self.output_size)
        # x = self.slidingFc(x)
        # x = x.view(b, -1)
        return x
        # return h, h_out, c_out   

slidingWindow = 50
seq_len = 10
InformationAdd = 0
InformationLack = 0
input_size = 10
status_size = 20
output_size = 10
model = CLSTMModel(input_size,status_size, output_size, InformationAdd, InformationLack).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

batch_size = 256
epochs = 20
delay = 10

pbar = tqdm(range(1, epochs+1))

u,y_seq = createDataset(input_size,status_size,output_size)
u = u.cuda()
y_seq = y_seq.cuda()
for epoch in pbar:
    # h0 = torch.randn(2, batch_size-slidingWindow, feature_num).cuda()
    # c0 = torch.randn(2, batch_size-slidingWindow, feature_num).cuda()
    for i in range(u.size()[0]-slidingWindow):
        input = u[i:i+slidingWindow]
        y = y_seq[i:i+slidingWindow]
        input = input.cuda()
        y = y.cuda()
        outputs = model(input)
        optimizer.zero_grad()
        loss = criterion(y,outputs)
        loss.backward()
        pbar.set_postfix({'train loss' : '{0:1.5f}'.format(loss)})
        optimizer.step()

100%|██████████| 20/20 [09:35<00:00, 28.78s/it, train loss=0.00874]


In [3]:
w = model.state_dict()["fc2.weight"]
C, D = w.split([status_size, input_size], 1)

In [4]:
print(C,D)

tensor([[-1.4827e+00,  3.0155e-02, -8.4355e-01, -2.3534e-01, -1.1329e-01,
          1.3857e-01, -1.7594e+00,  1.9898e-01, -5.9756e-01, -3.0593e-01,
         -1.5941e+00, -4.8806e-01,  8.4542e-01, -6.9222e-01,  7.0619e-01,
          8.0864e-01, -4.3364e-01,  1.3554e+00,  8.0440e-01,  9.5256e-01],
        [-3.8697e-01,  3.4242e-01, -1.1378e+00,  9.3182e-01,  5.1890e-01,
          1.2960e-03,  2.0555e-01,  8.4407e-02, -1.5373e+00,  9.7531e-01,
          1.0173e-01, -9.2757e-01, -9.5204e-01,  2.8672e-01,  3.4223e-01,
         -1.6956e-01,  2.2059e-01, -8.8120e-01, -1.9009e+00, -2.2415e-01],
        [-4.1541e-01, -4.7511e-01, -1.3011e+00, -6.6642e-02, -3.7115e-01,
          3.8658e-01, -1.2375e+00, -1.2711e+00, -1.4697e+00,  2.7861e-01,
         -1.0509e+00,  6.7557e-01, -4.1998e-02,  1.4125e+00, -6.0599e-01,
          8.1215e-01, -1.1208e-01,  6.2867e-01, -4.0436e-01, -3.3728e-01],
        [-1.4426e-01, -1.5524e+00, -1.9514e+00,  1.0499e+00,  2.8590e-01,
          5.4432e-01, -4.6377e-01, 